In [59]:
import pandas as pd 
import itertools 
import numpy as np

import pokemon_stats.mat_utils as mat

In [2]:
# read data
import importlib
importlib.reload(mat)
df = mat.importdf('single_single_type_chart')
df

,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
attack\defense,,,,,,,,,,,,,,,,,,
Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
Fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
Water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
Electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
Grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0
Ice,1.0,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
Fighting,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5
Poison,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0
Ground,1.0,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0


# iterate over single types to generate all pairwise permutations of dual types

In [65]:
single_types = df.index
n_single_types = len(single_types)

# include single types: normal/normal
repeat_types = [tuple([x,x]) for x in single_types] 
# repeat_types = list(zip(single_types, single_types))
# print(repeat_types)

# true duals with 2 distinct types
# order matters: water/fire and fire/water are both included
true_dual_types = list(itertools.permutations(single_types, 2))

# combine both lists
all_types = repeat_types + true_dual_types

# include single types: normal/normal
# order matters: water/fire and fire/water are both included
# dual_types = list(itertools.product(single_types, repeat=2))
# print(dual_types)

n_types = len(all_types)
print(n_dual_types)

multi_index = pd.MultiIndex.from_tuples(all_types, names=['first_type', 'second_type'])
# print(multi_index)

324


# Calculate values and store them in array

In [66]:
vals = np.zeros((n_types, n_types), dtype=float)

# choose attacking pokemon dual type
for i, atk_type in enumerate(all_types):

    j = 0
    # choose defending pokemon with single type
    for def_type in repeat_types:
        # cycle over attacking subtypes
        damages = df.loc[list(atk_type), def_type[0]]
        # take the max of these 2 attack values
        vals[i,j] = damages.max() 
        j += 1
    
    # choose defending pokemon with dual type
    for def_type in true_dual_types:
        # cycle over attacking subtypes
        damages = df.loc[list(atk_type), list(def_type)].prod(axis=1)
        max_damage = damages.max()
        vals[i,j] = damages.max()
        j += 1

#  Store the data into a data frame

In [68]:
# build the df once
df_dual_dual = pd.DataFrame(vals, index=multi_index, columns=multi_index)
df_dual_dual.index.name = 'attack\\defense'
df_dual_dual

first_type             Normal Fire Water Electric Grass  Ice Fighting Poison  \
second_type            Normal Fire Water Electric Grass  Ice Fighting Poison   
first_type second_type                                                         
Normal     Normal         1.0  1.0   1.0      1.0   1.0  1.0      1.0    1.0   
Fire       Fire           1.0  0.5   0.5      1.0   2.0  2.0      1.0    1.0   
Water      Water          1.0  2.0   0.5      1.0   0.5  1.0      1.0    1.0   
Electric   Electric       1.0  1.0   2.0      0.5   0.5  1.0      1.0    1.0   
Grass      Grass          1.0  0.5   2.0      1.0   0.5  1.0      1.0    0.5   
...                       ...  ...   ...      ...   ...  ...      ...    ...   
Fairy      Rock           1.0  2.0   1.0      1.0   1.0  2.0      2.0    1.0   
           Ghost          1.0  1.0   1.0      1.0   1.0  1.0      2.0    1.0   
           Dragon         1.0  1.0   1.0      1.0   1.0  1.0      2.0    1.0   
           Dark           1.0  1.0   1.0      1.0   1.0  1.0      2.0    1.0   
           Steel          1.0  0.5   1.0      1.0   1.0  2.0      2.0    1.0   

first_type             Ground Flying  ...  Fairy                             \
second_type            Ground Flying  ... Poison Ground Flying Psychic  Bug   
first_type second_type                ...                                     
Normal     Normal         1.0    1.0  ...    1.0    1.0    1.0     1.0  1.0   
Fire       Fire           1.0    1.0  ...    1.0    1.0    1.0     1.0  2.0   
Water      Water          2.0    1.0  ...    1.0    2.0    1.0     1.0  1.0   
Electric   Electric       0.0    2.0  ...    1.0    0.0    2.0     1.0  1.0   
Grass      Grass          2.0    0.5  ...    0.5    2.0    0.5     1.0  0.5   
...                       ...    ...  ...    ...    ...    ...     ...  ...   
Fairy      Rock           1.0    2.0  ...    1.0    1.0    2.0     1.0  2.0   
           Ghost          1.0    1.0  ...    1.0    1.0    1.0     2.0  1.0   
           Dragon         1.0    1.0  ...    0.5    1.0    1.0     1.0  1.0   
           Dark           1.0    1.0  ...    0.5    1.0    1.0     1.0  1.0   
           Steel          1.0    1.0  ...    2.0    2.0    2.0     2.0  2.0   

first_type                                           
second_type            Rock Ghost Dragon Dark Steel  
first_type second_type                               
Normal     Normal       0.5   0.0    1.0  1.0   0.5  
Fire       Fire         0.5   1.0    0.5  1.0   2.0  
Water      Water        2.0   1.0    0.5  1.0   1.0  
Electric   Electric     1.0   1.0    0.5  1.0   1.0  
Grass      Grass        2.0   1.0    0.5  1.0   0.5  
...                     ...   ...    ...  ...   ...  
Fairy      Rock         1.0   1.0    2.0  2.0   0.5  
           Ghost        1.0   2.0    2.0  2.0   1.0  
           Dragon       1.0   1.0    2.0  2.0   0.5  
           Dark         1.0   1.0    2.0  2.0   0.5  
           Steel        4.0   2.0    2.0  2.0   1.0  

[324 rows x 324 columns]

# Export

In [6]:
# save the transpoed and filled in version
mat.exportdf(df_dual_dual,"full_dual_dual_type_chart_from_single")

Saved as ../Input_data/full_dual_dual_type_chart_from_single.csv
